https://www.youtube.com/watch?v=ftMq5ps503w

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

In [2]:
X = pd.read_csv('sp500.csv')

In [3]:
X.shape

(4170, 1)

In [4]:
def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

In [5]:
def load_data(filename, seq_len, normalise_window):
    f = open(filename, 'r').read()
    data = f.split('\n')

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    
    if normalise_window:
        result = normalise_windows(result)

    result = np.array(result)
    
    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    #x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    #x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  
    #x_train = x_train.transpose()
    #x_test = y_test.transpose()
    return [x_train, y_train, x_test, y_test]

In [6]:
batch_size = 128
lstm_units = 100
seq_len = 50


X_train, y_train, X_test, y_test = load_data('sp500.csv', seq_len, True)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3709, 50), (3709,), (412, 50), (412,))

In [7]:
def get_batch(batch_size=64, X=X_train, y=y_train):
    n_batches = X.shape[0]//batch_size
    for i in range(n_batches):
        yield X[i*batch_size:(i+1)*batch_size, :].reshape(batch_size, -1, 1), y[i*batch_size : (i+1)*batch_size]

In [8]:
inputs = tf.placeholder(tf.float32, shape = [batch_size, 50, 1])
inputs_seq = tf.unstack(inputs)
outputs = tf.placeholder(tf.float32, shape = [batch_size])
learining_rate = tf.placeholder(tf.float32)

In [9]:
with tf.name_scope('rec_layer'):
    rnn_cell_1 = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(num_units = lstm_units)
    initial_state = rnn_cell_1.zero_state(batch_size=batch_size, dtype=tf.float32)
    lstm_output, state = tf.nn.dynamic_rnn(
        cell=rnn_cell_1, 
        inputs=inputs, 
        initial_state=initial_state)

In [10]:
with tf.name_scope('output_layer'):
    prediction = tf.contrib.layers.fully_connected(inputs=lstm_output[-1], num_outputs=1, activation_fn=None)
    loss = tf.reduce_sum(tf.square(prediction-outputs))

In [11]:
train = tf.train.AdamOptimizer(learning_rate=learining_rate).minimize(loss)

In [14]:
LEARMING_RATE = 0.05
epochs = 50

feed_dict = {learining_rate: LEARMING_RATE}

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    feeed_dict = {learining_rate: LEARMING_RATE}
    for epoch in range(epochs):
        batches = get_batch(batch_size)
        for batch in batches:
            feed_dict[inputs], feed_dict[outputs] = batch
            sess.run(train, feed_dict)
        print("Epoch: {}, loss: {}".format(epoch, sess.run(loss, feed_dict)))

Epoch: 0, loss: 47.188140869140625
Epoch: 1, loss: 48.11824035644531
Epoch: 2, loss: 44.94955062866211
Epoch: 3, loss: 47.43614959716797
Epoch: 4, loss: 51.9918098449707
Epoch: 5, loss: 48.801551818847656
Epoch: 6, loss: 46.10133361816406
Epoch: 7, loss: 45.77214050292969
Epoch: 8, loss: 45.40557098388672
Epoch: 9, loss: 45.02616882324219
Epoch: 10, loss: 44.60902404785156
Epoch: 11, loss: 44.36601257324219
Epoch: 12, loss: 44.382781982421875
Epoch: 13, loss: 44.47223663330078
Epoch: 14, loss: 44.501869201660156
Epoch: 15, loss: 44.49418258666992
Epoch: 16, loss: 44.477909088134766
Epoch: 17, loss: 44.45939636230469
Epoch: 18, loss: 44.44154357910156
Epoch: 19, loss: 44.42498016357422
Epoch: 20, loss: 44.41056442260742
Epoch: 21, loss: 44.39913558959961
Epoch: 22, loss: 44.391448974609375
Epoch: 23, loss: 44.38770294189453
Epoch: 24, loss: 44.38772964477539
Epoch: 25, loss: 44.39092254638672
Epoch: 26, loss: 44.39635467529297
Epoch: 27, loss: 44.402809143066406
Epoch: 28, loss: 44.4088